Relevant imports

In [ ]:
import random
from math import floor, ceil
import torch
from torch import nn, optim
from torch.utils import data
import numpy as np
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading data and split to train and validation sets

In [ ]:
# Read the data and compress it to 128*128 pixels
# Train set is a type of data set that contains all the images from both classifications
# with their labels
trainDataSet = ImageFolder("/content/drive/MyDrive/Colab Notebooks/dataCovid19", transform=Compose([Resize((128, 128)), ToTensor()]))
# Split to train and validation in ratio of 80:20
trainDataSet, validationDataSet = torch.utils.data.random_split(trainDataSet,
                                  [int(floor(len(trainDataSet)*0.80)), int(ceil(len(trainDataSet)*0.20))])


Mixup Augmentation

In [ ]:
# Get a data set and return a mixup image
class MixUpData(Dataset):
    def __init__(self, data_set):
        self.data_set = data_set
        self.mixup = []
        # Create the images of the mixup
        for i in range(8192):
            # Choose indexes of two randomally images
            idx1 = random.randint(0, len(self.data_set) - 1)
            idx2 = random.randint(0, len(self.data_set) - 1)

            # Get the vector of the images in the indexes
            image1 = self.data_set[idx1][0]
            image2 = self.data_set[idx2][0]

            # Create vectors for the labels (one hot vector)
            label1 = torch.zeros(2)
            label1[self.data_set[idx1][1]] = 1.
            label2 = torch.zeros(2)
            label2[self.data_set[idx2][1]] = 1.

            # Get a randomally lambda in a uniform distribution between 0.1 to 0.3
            # to get an extreme values
            lamb = np.random.uniform(0.1, 0.3, [1])[0]

            # Formula of mixup to get the new image and label
            mixImage = lamb * image1 + (1 - lamb) * image2
            mixLabel = lamb * label1 + (1 - lamb) * label2
            self.mixup.append([mixImage, mixLabel])

    def __len__(self):
        return len(self.mixup)

    def __getitem__(self, idx):
        return self.mixup[idx][0], self.mixup[idx][1]

Preparing the data for the learing models

In [ ]:
trainDataSet = MixUpData(trainDataSet)
# Prepare the train and validation sets to be an input for the NN
trainLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=64, shuffle=True)
validationLoader = torch.utils.data.DataLoader(validationDataSet, batch_size=64, shuffle=True)

Deep learning model - CNN

In [ ]:
# CNN parameters
cls = [16, 32, 64, 128]  # convolution layers size
krn = 5  # kernel size
std = 1  # stride
pad = 2  # num of padding
mpk = 2  # maxpool kernel
mps = 2  # maxpool stride


# CNN - deep learning method
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, cls[0], kernel_size=krn, stride=std, padding=pad),
            nn.BatchNorm2d(cls[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=mpk, stride=mps),
            nn.Dropout())
        self.layer2 = nn.Sequential(
            nn.Conv2d(cls[0], cls[1], kernel_size=krn, stride=std, padding=pad),
            nn.BatchNorm2d(cls[1]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=mpk, stride=mps),
            nn.Dropout())
        self.layer3 = nn.Sequential(
            nn.Conv2d(cls[1], cls[2], kernel_size=krn, stride=std, padding=pad),
            nn.BatchNorm2d(cls[2]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=mpk, stride=mps),
            nn.Dropout())
        self.layer4 = nn.Sequential(
            nn.Conv2d(cls[2], cls[3], kernel_size=krn, stride=std, padding=pad),
            nn.BatchNorm2d(cls[3]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=mpk, stride=mps),
            nn.Dropout())
        self.layer5 = nn.Sequential(
            nn.Linear(8192, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 2))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.reshape(x.size(0), -1)
        x = self.layer5(x)
        output = nn.functional.softmax(x, dim=1)
        return output

Non deep learning model - Perceptron

In [ ]:
# Perceptron - non deep learning method
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.perceptron = nn.Linear(49152, 2)

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.perceptron(x)
        output = nn.functional.softmax(x, dim=1)
        return output

Train the models and validation

In [ ]:
# Train the model
def train(model, optimizer):
    model.train()
    # Number of epochs
    for epoch in range(64):
        trainLoss = 0
        correctOutput = 0
        for batchIdx, (data, labels) in enumerate(trainLoader):
            optimizer.zero_grad()
            output = model(data)
            loss = nn.functional.mse_loss(output, labels)
            trainLoss += nn.functional.mse_loss(output, labels).item()
            prediction = output.max(1, keepdim=True)[1]
            true = labels.max(1, keepdim=True)[1]
            correctOutput += prediction.eq(true).cpu().sum()
            loss.backward()
            optimizer.step()


# Validation of the model
def validation(model):
    model.eval()
    validLoss = 0
    correctOutput = 0
    with torch.no_grad():
        for data, target in validationLoader:
            output = model(data)
            labelAsOneHot = []
            for i in target:
                tmp = [0, 0]
                tmp[i] = 1.
                labelAsOneHot.append(tmp)
            labelAsOneHot = torch.from_numpy(np.array(labelAsOneHot))
            validLoss += nn.functional.mse_loss(output, labelAsOneHot).item()
            prediction = output.max(1, keepdim=True)[1]
            correctOutput += prediction.eq(target.view_as(prediction)).cpu().sum()
    validLoss /= len(validationLoader.dataset)
    #print("Validation - Avg Loss: " + str(validLoss) + ", Accuracy: " + str(int(100. * correctOutput / len(validationLoader.dataset))) + "%")

Activate the models

In [ ]:
# Activate the perceptron model
model1 = Perceptron()
optimizer1 = optim.SGD(model1.parameters(), lr=0.1)
train(model1, optimizer1)
validation(model1)

# Activate the CNN model
model2 = CNN()
optimizer2 = optim.SGD(model2.parameters(), lr=0.1)
train(model2, optimizer2)
validation(model2)